In [1]:
%cd ~/gymnasium-test/
%load_ext line_profiler
%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

/home/n.saumik/gymnasium-test


In [1]:
import numpy as np
from tree import Tree, make_model, Forest, EarlyStopper
from tqdm import tqdm

importing numpy
importing tensorflow
done with imports


2024-10-31 01:39:20.613552: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["MPLCONFIGDIR"] = "/tmp/"
import tensorflow as tf

tf.config.set_visible_devices([], "GPU")

In [21]:
def get_diff(x):
    low = Forest(4, "crit-0-1-10-0.2", 0, x)
    low_X, low_Y = low.get_training_data("A", window=0)
    low_Y[:] = 0

    high = Forest(4, "crit-0-1-10-0.2", x, x * 2)
    high_X, high_Y = high.get_training_data("D", window=0)
    high_Y[:] = 1

    train_X = tf.convert_to_tensor(np.concatenate([low_X, high_X]))
    train_Y = tf.convert_to_tensor(np.concatenate([low_Y, high_Y]))

    model = make_model(4, 0, num_nodes=32, num_layers=2)
    steps = 65536

    ds = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
    ds = ds.repeat().shuffle(100000).batch(32).take(steps)
    for grp in tqdm(range(1), disable=True):
        acc = model.fit(ds, verbose=1).history["accuracy"][0]
        low_pred = model.predict(low_X)
        pcts = [(low_pred[:,1] <= pct).mean() for pct in np.linspace(0, 1, 11)]
        print(f"Steps {(grp+1)*steps}: {acc:.3f}")
        print(np.array(pcts).round(3))
        print()

In [22]:
%%time
get_diff(64)

65536/65536 [==============================] - 43s 653us/step - loss: 0.0267 - accuracy: 0.9921
Steps 65536: 0.992
[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

CPU times: user 55.5 s, sys: 3.38 s, total: 58.9 s
Wall time: 43.5 s


In [23]:
%%time
get_diff(256)

65536/65536 [==============================] - 42s 642us/step - loss: 0.1499 - accuracy: 0.9367
Steps 65536: 0.937
[0.    0.991 0.999 1.    1.    1.    1.    1.    1.    1.    1.   ]

CPU times: user 54.7 s, sys: 3.26 s, total: 58 s
Wall time: 42.8 s


In [24]:
%%time
get_diff(1024)

65536/65536 [==============================] - 43s 647us/step - loss: 0.5309 - accuracy: 0.7106
Steps 65536: 0.711
[0.    0.237 0.388 0.52  0.628 0.733 0.822 0.887 0.944 0.985 1.   ]

CPU times: user 55.3 s, sys: 3.46 s, total: 58.8 s
Wall time: 43.4 s


In [25]:
%%time
get_diff(4096)

65536/65536 [==============================] - 44s 665us/step - loss: 0.6447 - accuracy: 0.6130
Steps 65536: 0.613
[0.    0.037 0.149 0.292 0.483 0.726 0.88  0.957 0.99  1.    1.   ]

CPU times: user 57.7 s, sys: 3.68 s, total: 1min 1s
Wall time: 45.6 s


In [26]:
%%time
get_diff(16384)

65536/65536 [==============================] - 43s 655us/step - loss: 0.6662 - accuracy: 0.5916
Steps 65536: 0.592
[0.    0.005 0.076 0.213 0.515 0.854 0.967 0.996 1.    1.    1.   ]

CPU times: user 59.9 s, sys: 3.79 s, total: 1min 3s
Wall time: 48.5 s
